# Tweepy Streaming

In [2]:
import sys
sys.version

Out[ 1 ]: '3.5.2 (default, Sep 14 2017, 22:51:06) \n[GCC 5.4.0 20160609]'

##1.1. Instantiate OAuthHandler

In [4]:
url_rest      = "https://api.twitter.com/1.1/search/tweets.json" 
url_streaming = "https://stream.twitter.com/1.1/statuses/sample.json" 

Set up tokens and secrets.

In [6]:
consumer_key        = u'rfbbqTwbEdU7FfhEM5qhWdQSn'
consumer_secret     = u'AXbEXOoIOKIRPPOGXizYCY8zrPpg73aznPwqlbiPBtqyNjX1ZZ'
access_token        = u'50464413-petgMmE6qEKzW33wN7AdMMZODxME9KJSSADyKx99r' 
access_token_secret = u'JBVgy7RcTnHswLyIaOgkLNMRiIsrICBt7fB4CxFRd5Wra' 

Instantiate OAuthHandler function

In [8]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

#Error handling
try:
  redirect_url = auth.get_authorization_url()
except tweepy.TweepError:
  print ('Error! Failed to get request token.')

## 2.2. Stream Twitter data

Streaming or harvesting real time tweet data from Twitter is straighforward. When user makes request to stream, Twitter checks the authorization and opens streaming connection. When the connection is opened, Twitter server renders the tweets and user gets access to real time tweets.

![alt](https://cdn-images-1.medium.com/max/564/1*rPueOdOYwnsm8L3hST2ZbA.png)

`StreamListerner` function handles tweets that are received from the stream. This is a basic listener that just prints received tweets to stdout. The code below initiates list(`tweet_data`) and appends streaming tweets to the list.

In [13]:
from tweepy.streaming import StreamListener
from tweepy           import OAuthHandler
from tweepy           import Stream
from time             import strftime
from json             import loads

class StdOutListener(StreamListener):
  def __init__(self):
    self.tweet_data = []
    self.tweet_count = 0
        
  def on_data(self, data):
    self.tweet_count += 1
    print("Tweet", self.tweet_count, "downloaded")
    try:
      tweet = loads(data)
      self.tweet_data.append(tweet)
    except Exception as Argument:
      print ("Exception:\n", Argument)
    return True

  def on_error(self, status):
    print(status)

In [14]:
l = StdOutListener()

We use `Stream()` function streams the tweets in after authenticating our tokens. We can filter the tweets that contains certain words such as `trump`.

In [16]:
stream = Stream(auth, l)
stream.filter(track=['trump'])

In [17]:
tweets=l.tweet_data

Save `tweet_data` as pickle file so that we can access later.

In [19]:
import pickle
pickle.dump( l.tweet_data, open( "/dbfs/FileStore/tmp/stream_tweet_data.pkl", "wb" ) )

In [20]:
%sh ls -lh /dbfs/FileStore/tmp/*pkl

-rw-r--r-- 1 root root 300K Dec 8 2017 /dbfs/FileStore/tmp/all_data.pkl
-rw-r--r-- 1 root root 1.2M Dec 12 2017 /dbfs/FileStore/tmp/bitcoin_data.pkl
-rw-r--r-- 1 root root 2.2M Dec 12 2017 /dbfs/FileStore/tmp/cake_data.pkl
-rw-r--r-- 1 root root 3.3M Mar 14 18:45 /dbfs/FileStore/tmp/followers.pkl
-rw-r--r-- 1 root root 1.4M Mar 14 19:03 /dbfs/FileStore/tmp/following.pkl
-rw-r--r-- 1 root root 4.1M Mar 12 19:21 /dbfs/FileStore/tmp/geo_usa.pkl
-rw-r--r-- 1 root root 4.1M Mar 12 19:21 /dbfs/FileStore/tmp/geo-usa.pkl
-rw-r--r-- 1 root root 48M Mar 8 19:12 /dbfs/FileStore/tmp/geo)usa.pkl
-rw-r--r-- 1 root root 53M Mar 18 03:26 /dbfs/FileStore/tmp/google_tweets.pkl
-rw-r--r-- 1 root root 207K Mar 17 20:23 /dbfs/FileStore/tmp/hugo_follower_data.pkl
-rw-r--r-- 1 root root 82K Mar 17 20:24 /dbfs/FileStore/tmp/hugo_following_data.pkl
-rw-r--r-- 1 root root 102K Mar 17 20:32 /dbfs/FileStore/tmp/hugo_write_edgelist.pkl
-rw-r--r-- 1 root root 79M Mar 13 20:55 /dbfs/FileStore/tmp/russia_tweets.pkl
-rw-r--r-- 1 root root 6.5M Mar 18 20:28 /dbfs/FileStore/tmp/stream_tweet_data.pkl
-rw-r--r-- 1 root root 79M Mar 13 20:52 /dbfs/FileStore/tmp/trump_tweets.pkl
-rw-r--r-- 1 root root 3.3M Mar 17 20:12 /dbfs/FileStore/tmp/uyanga_followers.pkl
-rw-r--r-- 1 root root 1.4M Mar 17 20:13 /dbfs/FileStore/tmp/uyanga_following.pkl

Loading tweet data as `tweets`.

In [22]:
import pickle
tweets = pickle.load(open( "/dbfs/FileStore/tmp/stream_tweet_data.pkl", "rb" ) )

`Tweets` file is a list consists of dictionaries.

In [24]:
len(tweets), type(tweets), type(tweets[0])

Out[ 3 ]: (1125, list, dict)

Each dictionary contains several dictionary.

In [26]:
tweets[0]['text']

Out[ 10 ]: 'RT @goldengateblond: Can’t imagine this is legal or enforceable. Important point he still flatly refuses to acknowledge: Government employe…'

Following elements are single value elements. `Text` is the tweet itself,

In [28]:
for key, val in tweets[0].items():
  if type(val) in [int, str, bool] or type(val)==None:
    print(key)

id
quote_count
is_quote_status
lang
text
timestamp_ms
quoted_status_id_str
source
truncated
favorite_count
quoted_status_id
favorited
retweet_count
filter_level
reply_count
retweeted
id_str
created_at

In [29]:
for key, val in tweets[0].items():
  if type(val) not in [int, str, bool] or type(val)!=None:
    print(key)

in_reply_to_screen_name
reply_count
quoted_status_id
in_reply_to_user_id
contributors
favorite_count
quote_count
user
geo
quoted_status_id_str
place
coordinates
id
quoted_status
retweeted_status
entities
created_at
truncated
in_reply_to_user_id_str
source
id_str
retweet_count
filter_level
timestamp_ms
in_reply_to_status_id
retweeted
lang
favorited
text
is_quote_status
in_reply_to_status_id_str

In [30]:
import pandas as pd

col_names=['text', 'id', 'created_at', 'favourites_count', 'retweet_count','lang']
tweets_data = pd.DataFrame.from_records(tweets, columns=col_names)

In [31]:
tweets_data['favourites_count']

Out[ 25 ]: 
0 NaN
1 NaN
2 NaN
3 NaN
4 NaN
5 NaN
6 NaN
7 NaN
8 NaN
9 NaN
10 NaN
11 NaN
12 NaN
13 NaN
14 NaN
15 NaN
16 NaN
17 NaN
18 NaN
19 NaN
20 NaN
21 NaN
22 NaN
23 NaN
24 NaN
25 NaN
26 NaN
27 NaN
28 NaN
29 NaN
 ..
1095 NaN
1096 NaN
1097 NaN
1098 NaN
1099 NaN
1100 NaN
1101 NaN
1102 NaN
1103 NaN
1104 NaN
1105 NaN
1106 NaN
1107 NaN
1108 NaN
1109 NaN
1110 NaN
1111 NaN
1112 NaN
1113 NaN
1114 NaN
1115 NaN
1116 NaN
1117 NaN
1118 NaN
1119 NaN
1120 NaN
1121 NaN
1122 NaN
1123 NaN
1124 NaN
Name: favourites_count, dtype: float64

In [32]:
tweets[0]['entities']

Out[ 59 ]: 
{'urls': [],
 'symbols': [],
 'hashtags': [],
 'user_mentions': [{'screen_name': 'goldengateblond',
 'id': 15576928,
 'name': 'shauna',
 'indices': [3, 19],
 'id_str': '15576928'}]}

In [33]:
tweets[0]['retweeted_status']['text']

--------------------------------------------------------------------------- 
 KeyError Traceback (most recent call last)
 <command-184198> in <module> () 
 ----> 1 tweets [ 0 ] [ 'retweeted_status' ] [ 'text' ] 

 KeyError : 'retweeted_status'

In [34]:
tweets[0]['user'].keys()

Out[ 19 ]: dict_keys(['protected', 'name', 'screen_name', 'profile_use_background_image', 'listed_count', 'id', 'favourites_count', 'follow_request_sent', 'profile_text_color', 'translator_type', 'geo_enabled', 'profile_sidebar_fill_color', 'statuses_count', 'contributors_enabled', 'notifications', 'location', 'profile_background_image_url_https', 'profile_link_color', 'utc_offset', 'url', 'verified', 'default_profile', 'profile_sidebar_border_color', 'time_zone', 'id_str', 'friends_count', 'profile_image_url_https', 'lang', 'profile_background_color', 'default_profile_image', 'followers_count', 'description', 'profile_banner_url', 'profile_background_tile', 'created_at', 'profile_image_url', 'is_translator', 'profile_background_image_url', 'following'])

In [35]:
for key, val in tweets[0].items():
  if type(val) not in [int, str, bool]:
    if val!=None:
      print(key,":", val)

user : {'protected': False, 'name': 'FTID', 'screen_name': '4evaNFFC', 'profile_use_background_image': True, 'listed_count': 18, 'id': 249851209, 'favourites_count': 1008, 'follow_request_sent': None, 'profile_text_color': '333333', 'translator_type': 'none', 'geo_enabled': False, 'profile_sidebar_fill_color': 'DDEEF6', 'statuses_count': 13916, 'contributors_enabled': False, 'notifications': None, 'location': None, 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_link_color': '1DA1F2', 'utc_offset': None, 'url': None, 'verified': False, 'default_profile': True, 'profile_sidebar_border_color': 'C0DEED', 'time_zone': None, 'id_str': '249851209', 'friends_count': 636, 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/778115537645740033/0CvIo9pa_normal.jpg', 'lang': 'en', 'profile_background_color': 'C0DEED', 'default_profile_image': False, 'followers_count': 554, 'description': None, 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/249851209/1421617070', 'profile_background_tile': False, 'created_at': 'Wed Feb 09 22:35:02 +0000 2011', 'profile_image_url': 'http://pbs.twimg.com/profile_images/778115537645740033/0CvIo9pa_normal.jpg', 'is_translator': False, 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'following': None}
entities : {'urls': [], 'symbols': [], 'hashtags': [{'text': 'nffc', 'indices': [130, 135]}], 'user_mentions': []}

In [36]:
language=[]
index=[]
for i in range(len(tweets)):
  try: 
    if type(tweets[i]['lang'])==str:
      language.append(tweets[i]['lang'])
  except Exception as Argument:
    next
print(set(language))
print(len(set(language)))

{'it', 'und', 'in', 'es', 'en'}
5

In [37]:
index=[]
for i in range(len(tweets)):
  try: 
    if tweets[i]['lang']!='en':
      print(tweets[i]['retweeted_status']['text'])
      index.append(i)
  except Exception as Argument:
    next
print(index)

#SectorPrensaTecno TRUMP DEBATIÓ CON ESPECIALISTAS SOBRE LA VIOLENCIA EN VIDEOJUEGOS https://t.co/n0jRVaZs5Y El pre… https://t.co/vwnBJ1HGx8
Ma che dite sconsiderati? Se non ci fossero stati gli F35 come abrebbe fatto l'Italia a cacciare le navi da guerra… https://t.co/W2mcpDSEJW
URGENTE
Ditador norte-coreano convida Trump para partida de Bomba Patch https://t.co/lT4pQZr5Me
La industria siderúrgica española muestra su preocupación por los aranceles de Trump https://t.co/48Ddh6Mc9m
#LEADERSHIP 
#PeaceThroughStrength 
#MAGA 
#AmericaStrong 
#POTUS 
#ThankYouTrump 
@POTUS 
@realDonaldTrump https://t.co/gebDWI3YdD
#TheResistance #Resist #Resistance #Resistance2018 https://t.co/FisA9lC0sE
Since 1976 (Carter), here are *some* of the coups planned/carried out by the U.S.:

• Jamaica
• Chad
• Grenada
• Fi… https://t.co/a59P7Pu2uw
[16, 90, 103, 167, 171, 181, 201]

In [38]:
tweet=[]
for i in range(len(tweets)):
  try: 
    tweet.append(tweets[i]['retweeted_status']['text'])
  except Exception as Argument:
    next
print(len(tweet))
print(len(set(tweet)))

173
151

In [39]:
for i in range(len(tweets)):
  try:
    print(tweets[i]['retweeted_status']['created_at'])
  except Exception as Argument:
    print(Argument)

'retweeted_status'
'retweeted_status'
Sun Mar 11 08:00:31 +0000 2018
Sun Mar 11 12:40:40 +0000 2018
Sun Mar 11 14:48:57 +0000 2018
'retweeted_status'
'retweeted_status'
Sun Mar 11 15:23:38 +0000 2018
'retweeted_status'
'retweeted_status'
'retweeted_status'
'retweeted_status'
Sun Mar 11 03:31:10 +0000 2018
Sun Mar 11 15:19:34 +0000 2018
Sat Mar 10 15:56:43 +0000 2018
Sun Mar 11 07:13:17 +0000 2018
Sun Mar 11 15:30:30 +0000 2018
Wed Mar 07 12:55:01 +0000 2018
Sun Mar 11 02:40:51 +0000 2018
Sun Mar 11 15:29:00 +0000 2018
Sun Mar 11 15:30:17 +0000 2018
'retweeted_status'
Sun Mar 11 12:23:38 +0000 2018
'retweeted_status'
Sat Mar 10 22:20:36 +0000 2018
Sat Mar 10 03:09:38 +0000 2018
'retweeted_status'
Sat Mar 10 23:11:34 +0000 2018
Sun Mar 11 00:52:14 +0000 2018
Sun Mar 11 14:46:02 +0000 2018
Sun Mar 11 14:31:48 +0000 2018
Sun Mar 11 15:24:46 +0000 2018
Sun Mar 11 13:14:37 +0000 2018
Sun Mar 11 15:14:52 +0000 2018
Sun Mar 11 00:13:30 +0000 2018
Sun Mar 11 14:44:27 +0000 2018
Sun Mar 11 15:17:04 +0000 2018
Sun Mar 11 03:18:06 +0000 2018
'retweeted_status'
'retweeted_status'
'retweeted_status'
Sun Mar 11 15:27:49 +0000 2018
Sun Mar 11 15:19:02 +0000 2018
Sat Mar 10 21:11:55 +0000 2018
Sun Mar 11 08:34:29 +0000 2018
Sun Mar 11 14:08:31 +0000 2018
Wed Feb 28 01:50:12 +0000 2018
Sun Mar 11 15:26:06 +0000 2018
Sun Mar 11 02:40:51 +0000 2018
'retweeted_status'
Sun Mar 11 13:50:40 +0000 2018
'retweeted_status'
'retweeted_status'
Sun Mar 11 15:20:09 +0000 2018
Sun Mar 11 00:38:35 +0000 2018
Sun Mar 11 03:48:58 +0000 2018
Sun Mar 11 05:27:00 +0000 2018
'retweeted_status'
'retweeted_status'
Sun Mar 11 14:31:45 +0000 2018
Sun Mar 11 15:30:44 +0000 2018
Sun Mar 11 00:43:41 +0000 2018
Sun Mar 11 15:23:18 +0000 2018
Sun Mar 11 15:00:11 +0000 2018
'retweeted_status'
Sun Mar 11 15:00:42 +0000 2018
Sun Mar 11 15:28:17 +0000 2018
Sun Mar 11 07:13:17 +0000 2018
Sun Mar 11 01:58:36 +0000 2018
'retweeted_status'
Fri Mar 09 19:04:41 +0000 2018
Sun Mar 11 02:47:01 +0000 2018
Sun Mar 11 12:57:32 +0000 2018
Sun Mar 11 13:33:45 +0000 2018
Sun Mar 11 05:27:00 +0000 2018
Sat Mar 10 14:38:58 +0000 2018
Sat Mar 10 23:58:35 +0000 2018
'retweeted_status'
Sun Mar 11 15:29:04 +0000 2018
Sun Mar 11 05:01:09 +0000 2018
Sun Mar 11 15:06:01 +0000 2018
Sun Mar 11 15:28:29 +0000 2018
'retweeted_status'
Sun Mar 11 05:17:05 +0000 2018
Sun Mar 11 15:00:11 +0000 2018
Sun Mar 11 15:15:02 +0000 2018
'retweeted_status'
Sun Mar 11 15:02:36 +0000 2018
'retweeted_status'
Sun Mar 11 14:26:53 +0000 2018
Sun Mar 11 15:26:33 +0000 2018
'retweeted_status'
'retweeted_status'
Sun Mar 11 05:18:45 +0000 2018
Sun Mar 11 15:28:40 +0000 2018
Sun Mar 11 11:38:23 +0000 2018
Sun Mar 11 13:48:32 +0000 2018
Sun Mar 11 14:44:27 +0000 2018
Sun Mar 11 15:15:02 +0000 2018
Sun Mar 11 03:08:29 +0000 2018
Sun Mar 11 01:01:15 +0000 2018
Sun Mar 11 15:29:45 +0000 2018
Sun Mar 11 15:29:14 +0000 2018
Fri Mar 09 00:50:20 +0000 2018
Sun Mar 11 01:28:07 +0000 2018
Sun Mar 11 15:02:36 +0000 2018
Sun Mar 11 15:25:36 +0000 2018
'retweeted_status'
Sat Mar 10 22:01:31 +0000 2018
Sun Mar 11 15:28:29 +0000 2018
Sun Mar 11 00:41:26 +0000 2018
Sun Mar 11 07:00:49 +0000 2018
'retweeted_status'
Sun Mar 11 13:19:45 +0000 2018
Sun Mar 11 05:27:21 +0000 2018
Sun Mar 11 15:28:46 +0000 2018
Sun Mar 11 01:22:00 +0000 2018
Sun Mar 11 14:53:27 +0000 2018
Sun Mar 11 14:42:06 +0000 2018
Sun Mar 11 14:36:00 +0000 2018
'retweeted_status'
Sun Mar 11 03:45:07 +0000 2018
Sun Mar 11 12:37:53 +0000 2018
Sun Mar 11 15:20:09 +0000 2018
Sun Mar 11 01:38:37 +0000 2018
Sun Mar 11 01:20:57 +0000 2018
Fri Mar 09 22:28:12 +0000 2018
'retweeted_status'
Sun Mar 11 13:07:50 +0000 2018
Sun Mar 11 13:40:55 +0000 2018
Sun Mar 11 00:43:41 +0000 2018
'retweeted_status'
Sun Mar 11 10:44:23 +0000 2018
Sun Mar 11 04:46:15 +0000 2018
Sun Mar 11 01:10:24 +0000 2018
Sun Mar 11 02:40:51 +0000 2018
Sun Mar 11 15:29:14 +0000 2018
Sun Mar 11 15:28:45 +0000 2018
'retweeted_status'
Sat Mar 10 20:02:00 +0000 2018
'retweeted_status'
Sun Mar 11 15:28:07 +0000 2018
Sat Ma

In [40]:
twitters=[]
for i in range(len(tweets)):
  try: 
    twitters.append(tweets[i]['retweeted_status']['user']['screen_name'])
  except Exception as Argument:
    next
print(len(twitters))
print(len(set(twitters)))

173
138

In [41]:
twitters_fol=[]
for i in range(len(tweets)):
  try: 
    twitters_fol.append(tweets[i]['retweeted_status']['user']['followers_count'])
  except Exception as Argument:
    next
print(len(twitters_fol))
print(len(set(twitters_fol)))

173
150

__Uyanga__: The following command doesn't work. It may be that not all tweets have the `'retweeted_status'` field (likely as not all tweets are retweets.) I reran the code above and so the `tweets` list has changed.